# Impact Aware Embeddings
- reading textual and price datasets
- labeling the price dataset and merging with textual dataset
- finetuning the embedding model on labelings to make it impact aware
- evaluating the results and comparing with normal embeddings